In [89]:
#Import Libraries
from bs4 import BeautifulSoup
from html2text import html2text
import pandas as pd
import pprint as pp
import requests
import time

In [90]:
#Extract Job URL Function
def extract_job_url_from_result(soup): 
    urls = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            this_url = a['href']
            to_go_url = "https://www.indeed.com/viewjob" + this_url[7:]
            urls.append(to_go_url)
    return(urls)

In [91]:
#Extract Job Title & Description
def extract_text_from_jobURL(url):
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    #Retrive the job title
    b = soup.find('b','jobtitle')
    if b != None:
        
        #Retrive the job description
        table = soup.find('table', id = 'job-content')
        span = table.find('span', id = 'job_summary')

        div = span.find('div')

        #Because in some posts there isn't exist the div class
        if div==None:
            div=span

        #Cleaning (remove newllines, * , spaces)
        b=html2text(str(b)).replace(" ", "").replace("*","").replace("\n","")
        div=html2text(str(div)).replace(" ", "").replace("*","").replace("\n","")
    
        return(b,div)
    
    else:
        
        return([])

In [92]:
#Import the Job Title Dataset
job_titles=pd.read_csv("..\Datasets\Job_Titles.csv", sep = ",")

#Create job_titles list
job_list=list(job_titles.Job_titles)

#Replace spaces with '+'
job_list2=[]
for item in job_list:
    job_list2.append(item.replace(' ', '+'))
    
job_list=job_list2

In [93]:
#Create URLs

#Initialize the url job list and city list
url_list=[]
city_list=['New+York','England']

for city in city_list:
    for job in job_list:
        for start in range(0,20,10):
            URL="http://www.indeed.com/jobs?q="+ job + "%2420%2C000&l=" + str(city) + "&start=" + str(start)
            url_list.append(URL)         

In [94]:
#Main loop

#Initialize the dataframe
columns=["job_title","job_description"]
df = pd.DataFrame(columns = columns)
n=0

#Extract the information from the site
for i in url_list:

    #Conducting a request of the stated URL above:
    page = requests.get(i)
    
    #Add 1 second sleep
    time.sleep(0.3)
    
    #Specifying a desired format of “page” using the html parser
    soup = BeautifulSoup(page.text, "html.parser")

    #Go to job page
    extract_job=extract_job_url_from_result(soup)
    
    #Check if there is a job in indeed
    if extract_job !=[]:
        job_url = extract_job[0]
        
        #Extract job description (remove newllines, * , spaces)
        job_description=extract_text_from_jobURL(job_url)

        #Add a new row
        if job_description !=[]:
            df.loc[len(df)]=[job_description[0],job_description[1]]

In [95]:
#Create a csv file
df.to_csv("Initial_Job_Df.csv")